In [1]:
#import modules
import numpy as np
import pandas as pd
import re
import itertools
import glob
from functools import reduce
from itertools import groupby
import primer3

#Snapgene module
from snapgene_reader import snapgene_file_to_dict, snapgene_file_to_seqrecord

#Functions
def band_filter(distance):
    accuracy_of_fragment_analzyer = 0.1
    flag = 1

    if max(distance) > 9500:
        flag = flag*0
    else:
        flag = flag*1
    
    if min(distance) < 100:
        flag = flag*0
    else:
        flag = flag*1
    
    for i in range(len(distance)-1):
        band_size_diff = distance[i+1] - distance[i]
        if band_size_diff > accuracy_of_fragment_analzyer*distance[i]:
            flag = flag*1
        else:
            flag = flag*0

    if flag == 1:
        return True
    else: 
        return False

def list_sort(curr_list): 
    curr_list.sort(key = lambda x: x[0]) 
    return pd.DataFrame(curr_list,columns = ['fragment_number','dna_sequence'])

def percent_gc(seq):
    gc = 0
    for i in range(len(seq)):
        if seq[i] == 'G' or seq[i]=='g':
            gc = gc + 1
        if seq[i] == 'C' or seq[i]=='c':
            gc = gc + 1

    GC = gc*100/len(seq) 
    return GC

def revcom(seq):
    answer=''
    for i in range(0,len(seq)):       
        if seq[i]== 'A' or seq[i]== 'a':
            answer ='T'+answer
        elif seq[i]=='C' or seq[i]=='c':
            answer ='G'+answer
        elif seq[i]=='G' or seq[i]=='g':
            answer ='C'+answer
        elif seq[i]=='T' or seq[i]=='t':
            answer ='A'+answer
    return answer

def palindrome(n):
    return n == n[::-1]

def sim_score(seq1,seq2):
    return sum (seq1[i] == seq2[i] for i in range(len(seq1)))

def freq_count(x):
    unique, counts = np.unique(x, return_counts=True)
    y = np.asarray((unique, counts)).T[::-1] 
    return y

def comb_off_target(a,b):
    c = []
    for i in range(len(a)):
        c_flag = 0
        for j in range(len(b)):
            if a[i][0] == b[j][0]:
                c.append([a[i][0],a[i][1] + b[j][1]])   
                c_flag = 1
        if c_flag == 0:
            c.append([a[i][0],a[i][1]])

    for i in range(len(b)):
        c_flag = 0
        for j in range(len(a)):
            if b[i][0] == a[j][0]:
                c_flag = 1
        if c_flag == 0:
            c.append([b[i][0],b[i][1]])      
    
    return c

def off_target_check(x,hm_score):
    flag_ot = 1
    if x[0][0] == 22:
        if x[0][1] > 1:
            flag_ot = flag_ot*0
        else:
            flag_ot = flag_ot*1
    if x[1][0] >= hm_score: 
        flag_ot = flag_ot*0
    else:
        flag_ot = flag_ot*1
        
    return flag_ot

def three_frag_score(seq1,seq2,large_3f_score,mid_3f_score):
    if sim_score(seq1,seq2) >= large_3f_score: 
        mid_seq1 = seq1[8:14]
        mid_seq2 = seq2[8:14]
        if sim_score(mid_seq1,mid_seq2) >= mid_3f_score:
            return 0
        else:
            return 1
    else:
        return 1
    
def mismatch_score(seq1,seq2):
    score = 0
    for a in range(len(seq1)):
        if seq1[a] == seq2[a]:
            score = score + (5.5-a)*(5.5-a)
        elif seq1[a] == 'T' and seq2[a] == 'C':
            score = score + 0.75*(5.5-a)*(5.5-a)
        elif seq1[a] == 'G' and seq2[a] == 'A':
            score = score + 0.75*(5.5-a)*(5.5-a)
            
    return score

def overlap(a, b):
    return max(i for i in range(len(b)+1) if a.endswith(b[:i]))

def getTM(seq1, seq2):
    tm1 = primer3.calcTm(seq1)
    tm2 = primer3.calcTm(seq2)

    return [tm1, tm2]

def combine_strings(x):
    for perm in permutations(x):
        linked = [perm[i][:-5] for i in range(len(perm)-1) 
                               if perm[i][-5:]==perm[i+1][:5]]
        if len(perm)-1==len(linked):
            return "".join(linked)+perm[-1]
    return None

def CntSubstr(pattern, string):
    a = [m.start() for m in re.finditer(
        '(?={0})'.format(re.escape(pattern)), string)]
    return a[0]

def is_valid_guide(seq1):
    flag = 1
    curr_target = seq1
    guide = curr_target[1:23]
    guide_u = revcom(curr_target[1:15])
    guide_l = curr_target[9:23]
    
    #Intrafragment Analysis
    #criteria: GC content
    if gc_stringency == 0:
        gc_crit = -15
    elif gc_stringency == 1:
        gc_crit = -10
    elif gc_stringency == 2:
        gc_crit = -5
    else:
        gc_crit = 0 
        
    if pfago_flag == 0:
        gc = percent_gc(curr_target) 
        if gc > 15 and gc <= 60 + gc_crit: #45,50,55,60
            flag = flag*1
        else: 
            flag = flag*0

    else:
        gc = percent_gc(curr_target)
        if gc > 30 and gc <= 75 + gc_crit:
            flag = flag*1
        else: 
            flag = flag*0
        
    if pfago_flag == 0:
        gc = percent_gc(guide_u)
        if gc > 15 and gc <= 65 + gc_crit: #50,55,60,65
            flag = flag*1
        else: 
            flag = flag*0

    else:
        gc = percent_gc(guide_u)
        if gc > 30 and gc <= 75 + gc_crit:
            flag = flag*1
        else: 
            flag = flag*0
    
    if pfago_flag == 0:
        gc = percent_gc(guide_l)
        if gc > 15 and gc <= 65 + gc_crit:
            flag = flag*1
        else: 
            flag = flag*0

    else:
        gc = percent_gc(guide_l)
        if gc > 30 and gc <= 75 + gc_crit:
            flag = flag*1
        else: 
            flag = flag*0
    
    if flag == 0:
        #print('Problem due to GC content')
        return False
    
    #criteria: G-quadraplex or more than 5 consecutive A or Ts 
    ## G-quadraplex
    if curr_target.find('GGGG') >-1: 
        flag = flag*0
    else: 
        flag = flag*1

    if curr_target.find('CCCC') >-1:
        flag = flag*0
    else: 
        flag = flag*1

    ##more than 5 consecutive A or Ts 
    if curr_target.find('A'*AT_stringency) >-1:
        flag = flag*0
    else: 
        flag = flag*1

    if curr_target.find('T'*AT_stringency) >-1:
        flag = flag*0
    else: 
        flag = flag*1
    
    if flag == 0:
        #print('Problem due to GGGG/TTTT/AAAAAA/TTTTTT')
        return False
    
    
    #Interfragment Analysis
    #Off-target scores[Constraint 1]: on 22 bp
    g_off_target_str_plasmid_score = []
    for o_t in range(len(off_target_str_plasmid_seq)):
        g_off_target_str_plasmid_score.append(sim_score(guide,off_target_str_plasmid_seq[o_t]))
    
     
    g_off_target_op_str_plasmid_score = []
    for o_t in range(len(off_target_op_str_plasmid_seq)):
        g_off_target_op_str_plasmid_score.append(sim_score(guide,off_target_op_str_plasmid_seq[o_t]))

    g_str_freq_count = freq_count(g_off_target_str_plasmid_score)
    g_op_str_freq_count = freq_count(g_off_target_op_str_plasmid_score)
    g_freq_count = comb_off_target(g_str_freq_count,g_op_str_freq_count)

    if off_target_check(g_freq_count,hm_score) == 1:
        flag = flag*1
    else:
        flag = flag*0
    
    if flag == 0:
        #print('Problem due to homology score')
        return False
    
    #Off-target scores[Constraint 2]: Three fragment check
    g_3_frag_target_str_plasmid_score = []
    for f_t in range(len(off_target_str_plasmid_seq)):
        g_3_frag_target_str_plasmid_score.append(three_frag_score(guide,off_target_str_plasmid_seq[f_t],large_3f_score,mid_3f_score))

    g_3_frag_target_op_str_plasmid_score = []
    for f_t in range(len(off_target_op_str_plasmid_seq)):
        g_3_frag_target_op_str_plasmid_score.append(three_frag_score(guide,off_target_op_str_plasmid_seq[f_t],large_3f_score,mid_3f_score))

    if len(g_3_frag_target_str_plasmid_score) - sum(g_3_frag_target_str_plasmid_score) < 2:
        flag = flag*1
    else:
        flag = flag*0

    if len(g_3_frag_target_op_str_plasmid_score) - sum(g_3_frag_target_op_str_plasmid_score) < 2:
        flag = flag*1
    else:
        flag = flag*0
    
    if flag == 0:
        #print('Problem due to 3 frag score')
        return False
    
    #palindromic sticky ends
    sticky_end_target = curr_target[6:18]
    if palindrome(sticky_end_target):
        flag = flag*0
    else:
        flag = flag*1 
        
    #valid guide or not
    if flag == 1:
        return True
    else: 
        #print('Problem due to sticky ends')
        return False

#Restriction Enzyme for Verification
df = pd.read_excel("common_re_list.xlsx")
re_site = df.to_numpy()
count = 0
for re_i in range(1,np.shape(re_site)[0]):
    curr_re_seq = re_site[re_i,0]
    if curr_re_seq == revcom(curr_re_seq):
        count = count + 1
        
if count == len(re_site)-1:
    print("All restriction enzymes are palindromic") 
    
final_re_count = 0
working_plasmid_count = 0
common_final_re_df_index = ['RE Combinations']
final_fragments = []
final_interm_fragments = []
for files in glob.glob("*.dna"):
    snap_file = files  
    
    #reading snap_gene_file
    dictionary = snapgene_file_to_dict(snap_file)
    seqrecord = snapgene_file_to_seqrecord(snap_file)
    filename = snap_file.split('.')[0] + '.xlsx'

    count = 0
    data_stat = []
    for i in range(len(dictionary['features'])):
        if 'fragment' in dictionary['features'][i]['name']:
            count = count + 1
            frag_start, frag_end = next(iter(dictionary['features'][i]['segments'][0].items()))[1].split('-')
            f_start = int(frag_start)-1
            f_end = int(frag_end)
            if f_start < f_end:
                curr_seq = dictionary['seq'][f_start:f_end]
            else:
                curr_seq = dictionary['seq'][f_start:len(dictionary['seq'])] + dictionary['seq'][0:f_end]
            data_stat.append([int(dictionary['features'][i]['name'].replace('fragment','')),curr_seq])
            
        if 'Fragment' in dictionary['features'][i]['name']:
            count = count + 1
            frag_start, frag_end = next(iter(dictionary['features'][i]['segments'][0].items()))[1].split('-')
            f_start = int(frag_start)-1
            f_end = int(frag_end)
            if f_start < f_end:
                curr_seq = dictionary['seq'][f_start:f_end]
            else:
                curr_seq = dictionary['seq'][f_start:len(dictionary['seq'])] + dictionary['seq'][0:f_end]
            data_stat.append([int(dictionary['features'][i]['name'].replace('Fragment','')),curr_seq])

    df = list_sort(data_stat)
    insert_order_specified = np.array(df.fragment_number)
    sequences_specified = np.array(df.dna_sequence)

    order = insert_order_specified.argsort()
    insert_order = insert_order_specified[order]
    sequences_case = sequences_specified[order]

    pre_sequences = []
    for i in range(len(sequences_case)):
        pre_sequences.append(sequences_case[i].upper())
    
    del_index = []
    merge_flag = 0
    merging_tm_template = []
    for i in range(len(pre_sequences)):
        if len(pre_sequences[i])<80:
            del_index.append(i)
            print('fragment less than 80 bp')
            merge_flag = 1
            if len(pre_sequences[i])%2 == 0:
                first_half = pre_sequences[i][0:len(pre_sequences[i])//2]
                second_half = pre_sequences[i][len(pre_sequences[i])//2:]
            else:
                first_half = pre_sequences[i][0:len(pre_sequences[i])//2]
                second_half = pre_sequences[i][len(pre_sequences[i])//2:]

            if i==0:
                merging_tm_template.append([len(pre_sequences)-1,i+1])     

                pre_sequences[len(pre_sequences)-1] = pre_sequences[len(pre_sequences)-1] + first_half
                pre_sequences[i+1] = second_half + pre_sequences[i+1]

            elif i==len(pre_sequences)-1: 
                merging_tm_template.append([i-1,0])   

                pre_sequences[i-1] = pre_sequences[i-1] + first_half
                pre_sequences[0] =  second_half + pre_sequences[0]
            else:
                merging_tm_template.append([i-1,i+1])   

                pre_sequences[i-1] = pre_sequences[i-1] + first_half
                pre_sequences[i+1] = second_half + pre_sequences[i+1]

    sequences = []
    if len(del_index) > 0:
        for i in range(len(pre_sequences)):
            if len(pre_sequences[i]) >= 80:
                sequences.append(pre_sequences[i])
    else:
        sequences = pre_sequences

    #Restoring presequence (by removing if there was any addition)
    if merge_flag == 1:
        pre_sequences = []
        for i in range(len(sequences_case)):
            pre_sequences.append(sequences_case[i].upper())

    ##Complete Plasmid Sequence
    for p_i in range(len(sequences)):
        if p_i == 0:
            plasmid_seq = sequences[p_i]
        else:
            plasmid_seq = plasmid_seq + sequences[p_i]

    re_plasmid_seq = plasmid_seq + sequences[0][0:7]
    str_plasmid_seq = plasmid_seq + sequences[0][0:21] #to ensure we analyze circular plasmid 
    op_str_plasmid_seq = revcom(str_plasmid_seq)

    #Off-target library
    off_target_str_plasmid_seq = []
    for i in range(len(plasmid_seq)):
        off_target_str_plasmid_seq.append(str_plasmid_seq[0+i:22+i])
    
    off_target_op_str_plasmid_seq = []
    for i in range(len(plasmid_seq)):
        off_target_op_str_plasmid_seq.append(op_str_plasmid_seq[0+i:22+i])

    #Guide Library
    search_space = '80 bp fragment'
    guide_24bp = []
    pre_mmlig_guide = []
    for i in range(len(sequences)):
        guide_24bp.append('guide for joint'+str(i))
        pre_mmlig_guide.append([i,'guide for joint'+str(i)])
        if i == 0:
            search_space = np.vstack((search_space,sequences[len(sequences)-1][-40:]+sequences[i][0:40]))
        else:
            search_space = np.vstack((search_space,sequences[i-1][-40:]+sequences[i][0:40]))

    for i in range(57):
        curr_guides = []
        for j in range(len(sequences)):
            curr_guides.append(search_space[j+1][0][0+i:24+i])

        guide_24bp = np.vstack((guide_24bp,curr_guides))

    #guide library centered
    c_guide_24bp = guide_24bp[0,:]
    k = 29
    for i in range(1,np.shape(guide_24bp)[0]):
        c_guide_24bp = np.vstack((c_guide_24bp,guide_24bp[k,:]))
        if i%2 == 0:
            k = k - i
        else:
            k = k + i

    guide_library = np.asarray(c_guide_24bp)

    #Step 1: measuring fragments and guide search space GC and deciding which PfAgo to use
    inserts_percent_gc = []
    for i in range(len(sequences)):
        inserts_percent_gc.append(percent_gc(sequences[i]))

    guide_search_space_gc = []
    for i in range(1, np.shape(search_space)[0]):
        guide_search_space_gc.append(percent_gc(search_space[i][0]))
    
    if sum(i > 57 for i in inserts_percent_gc)/len(inserts_percent_gc) > 0.5:
        if any(gc_ss > 65 for gc_ss in guide_search_space_gc):
            pfago_flag = 1
            print('Use mutant PfAgo for plasmid assembly')
        else:
            pfago_flag = 0
            print('Use wild-type + mutant PfAgo for plasmid assembly')
    else:
        pfago_flag = 0
        print('Use wild-type + mutant PfAgo for plasmid assembly')
    
    param_space = []
    for k in range(2):
        for i in range(3):
            for j in range(i,i+2):
                param_space.append([15+i,13+j,5+k])

    for at_repeats_i in range(2):
        for gc_content_i in range(4):
            hm_score = param_space[0][0]
            large_3f_score = param_space[0][1]
            mid_3f_score = param_space[0][2]
            score_param_space = 1

            insert_target_found = []
            m_check = []
            for i in range(len(sequences)):
                insert_target_found.append(False)
                m_check.append(-1)

            targets_found = False

            final_guide = []
            AT_stringency = 5 + at_repeats_i
            gc_stringency = gc_content_i

            while not targets_found:
                for m in range(len(sequences)):
                    if not insert_target_found[m]:
                        for i in range(1,np.shape(guide_library)[0]):
                            if i > m_check[m]:
                                target = guide_library[i,m]
                                m_check[m] = i

                                # All criteria except mismatch ligation are checked here
                                if is_valid_guide(target):       
                                    pre_mmlig_guide[m][1] = target
                                    insert_target_found[m] = True
                                    break

                if all(insert_target_found):
                    targets_found = True
                else: 
                    targets_found = False 

                #mismatch ligation check
                if targets_found:
                    sticky_end = []
                    for q in range(len(sequences)):
                        sticky_end.append(pre_mmlig_guide[q][1][6:18])
                        sticky_end.append(revcom(pre_mmlig_guide[q][1][6:18]))

                    mmlig_matrix = np.zeros([len(sticky_end),len(sticky_end)])
                    for mml1 in range(len(sticky_end)):
                        for mml2 in range(len(sticky_end)):
                            if mml1 != mml2:
                                mmlig_matrix[mml1,mml2] = mismatch_score(sticky_end[mml1],sticky_end[mml2])

                    if np.all(mmlig_matrix < 100): 
                        final_guide = pre_mmlig_guide
                    else:
                        max_ind = np.unravel_index(np.argmax(mmlig_matrix, axis=None), mmlig_matrix.shape)
                        m_mml1 = max_ind[0]//2 
                        m_mml2 = max_ind[1]//2 
                        if m_mml1 == m_mml2:
                            m_mml = m_mml1
                        else:
                            if m_check[m_mml1] < m_check[m_mml2]:
                                m_mml = m_mml1
                            else:
                                m_mml = m_mml2
                        targets_found = False
                        insert_target_found[m_mml] = False

                if len(param_space) > score_param_space:
                    if any(x==57 for x in m_check):
                        hm_score = param_space[score_param_space][0]
                        large_3f_score = param_space[score_param_space][1]
                        mid_3f_score = param_space[score_param_space][2]
                        score_param_space = score_param_space + 1
                        insert_target_found = [] 
                        m_check = []
                        for i in range(len(sequences)):
                            insert_target_found.append(False)
                            m_check.append(-1)
                else:
                    break

            if targets_found:
                break

        if targets_found:
            break
            
    if not targets_found:
        print('No guides can be found for ' + snap_file)
        to_find_primer_flag = 0
    else:
        print('Guides obtained successfully for ' + snap_file)
        to_find_primer_flag = 1
        working_plasmid_count = working_plasmid_count + 1
        
    ini_tm_len = 18
    if to_find_primer_flag == 1:
        
        #Small fragment merging primers
        merging_primers = ['Temp Number', 'F Primer', 'R Primer','Extension Time (in secs)', 'Template Length']
        if merge_flag == 1:
            for i in range(np.shape(merging_tm_template)[0]):
                cb_region = ['Temp Number', 'F Primer', 'R Primer','Extension Time (in secs)']
                for j in range(np.shape(merging_tm_template)[1]):
                    curr_temp = merging_tm_template[i][j]

                    tm_flag = 0
                    
                    ini_mg_fp_seq = pre_sequences[curr_temp][0:ini_tm_len]
                    if percent_gc(ini_mg_fp_seq) < 30:
                        tm_len = 25
                    elif percent_gc(ini_mg_fp_seq) > 70:
                        tm_len = 18
                    else:
                        tm_len = 20
                            
                    mg_fp_seq = pre_sequences[curr_temp][0:tm_len]
                    
                    ini_mg_rp_seq = revcom(pre_sequences[curr_temp][-ini_tm_len:])
                    if percent_gc(ini_mg_rp_seq) < 30:
                        tm_len = 25
                    elif percent_gc(ini_mg_rp_seq) > 70:
                        tm_len = 18
                    else:
                        tm_len = 20
                        
                    mg_rp_seq = revcom(pre_sequences[curr_temp][-tm_len:])
                            
                    while tm_flag == 0:
                        ta_tm_values = getTM(mg_fp_seq,mg_rp_seq)
                        if np.abs(ta_tm_values[0]-ta_tm_values[1]) > 2:
                            tm_flag = 0
                            tm_len = tm_len + 1 
                            if ta_tm_values[0] - ta_tm_values[1] > 0:
                                mg_rp_seq = revcom(pre_sequences[curr_temp][-tm_len:])
                            else:
                                mg_fp_seq = pre_sequences[curr_temp][0:tm_len]
                        else:
                            tm_flag = 1
                            cb_region = np.vstack((cb_region, [curr_temp, mg_fp_seq, mg_rp_seq, np.round(len(pre_sequences[curr_temp])*(30/1000),2)]))

                #dividing small fragment < 80 bp and adding it to adjacent fragments
                if cb_region[1][0] == len(pre_sequences) - 1:
                    small_insert_seq = pre_sequences[len(pre_sequences) - 1]

                elif cb_region[1][0] == 0:
                    small_insert_seq = pre_sequences[0]

                else:
                    small_insert_seq = pre_sequences[int(cb_region[1][0]) + 1]

                first_half = small_insert_seq[0:len(small_insert_seq)//2]
                second_half = small_insert_seq[len(small_insert_seq)//2:]

                for i in range(2):
                    if i == 0: 
                        merging_primers = np.vstack((merging_primers,[cb_region[i+1][0],cb_region[i+1][1],revcom(first_half) + cb_region[1][2],cb_region[i+1][3],len(first_half)+len(pre_sequences[int(cb_region[i+1][0])])]))
                    else:
                        merging_primers = np.vstack((merging_primers,[cb_region[i+1][0],second_half + cb_region[2][1],cb_region[i+1][2],cb_region[i+1][3],len(second_half)+len(pre_sequences[int(cb_region[i+1][0])])]))

        #Primers for One pot PCR fragments
        PCR_primers = ['Temp Number', 'F Primer', 'R Primer','Extension Time (in secs)', 'Template Length']
        for i in range(np.shape(final_guide)[0]):
            cb_region = ['Temp Number', 'F Primer', 'R Primer','Extension Time (in secs)']
            if i == np.shape(final_guide)[0] - 1:
                curr_guide_1 = final_guide[i][1]
                curr_guide_2 = final_guide[0][1]
            else:
                curr_guide_1 = final_guide[i][1]
                curr_guide_2 = final_guide[i+1][1]

            f_rule = 0
            r_rule = 0

            #F primer
            if i == 0:
                f_temp_left = sequences[len(sequences) - 1]
                f_temp_right = sequences[i]
            else:
                f_temp_left = sequences[i-1]
                f_temp_right = sequences[i]

            if curr_guide_1 in f_temp_left:
                fg_ind  = f_temp_left.index(curr_guide_1)
                overhang_f = f_temp_left[fg_ind:len(f_temp_left)]
                
                if percent_gc(f_temp_right[0:ini_tm_len]) < 30 :
                    tm_len = 25
                elif percent_gc(f_temp_right[0:ini_tm_len]) > 70:
                    tm_len = 18
                else:
                    tm_len = 20
                            
                f_primer = f_temp_right[0:tm_len]
                f_rule = 1

            elif curr_guide_1 in f_temp_right:
                fg_ind = f_temp_right.index(curr_guide_1)
                
                if percent_gc(f_temp_right[fg_ind:fg_ind+ini_tm_len]) < 30 :
                    tm_len = 25
                elif percent_gc(f_temp_right[fg_ind:fg_ind+ini_tm_len]) > 70:
                    tm_len = 18
                else:
                    tm_len = 20
                            
                f_primer = f_temp_right[fg_ind:fg_ind+tm_len]
                overhang_f = ''
                f_rule = 2

            else: #present on both fragments
                join_temp = f_temp_left + f_temp_right
                fg_ind = join_temp.index(curr_guide_1)
                overhang_f = f_temp_left[fg_ind:len(f_temp_left)]
                
                if percent_gc(f_temp_right[0:ini_tm_len]) < 30 :
                    tm_len = 25
                elif percent_gc(f_temp_right[0:ini_tm_len]) > 70:
                    tm_len = 18
                else:
                    tm_len = 20
                    
                f_primer = f_temp_right[0:tm_len]
                f_rule = 3

            #R primer
            if i == len(sequences) - 1:
                r_temp_left = sequences[len(sequences) - 1]
                r_temp_right = sequences[0]
            else:
                r_temp_left = sequences[i]
                r_temp_right = sequences[i+1]

            if curr_guide_2 in r_temp_left:
                rg_ind  = r_temp_left.index(curr_guide_2)
                
                if percent_gc(revcom(r_temp_left[rg_ind+24-ini_tm_len:rg_ind+24])) < 30 :
                    tm_len = 25
                elif percent_gc(revcom(r_temp_left[rg_ind+24-ini_tm_len:rg_ind+24])) > 70:
                    tm_len = 18
                else:
                    tm_len = 20
                        
                r_primer = revcom(r_temp_left[rg_ind+24-tm_len:rg_ind+24])
                overhang_r = ''
                r_rule = 1

            elif curr_guide_2 in r_temp_right:
                rg_ind  = r_temp_right.index(curr_guide_2)
                
                if percent_gc(revcom(r_temp_left[-ini_tm_len:])) < 30 :
                    tm_len = 25
                elif percent_gc(revcom(r_temp_left[-ini_tm_len:])) > 70:
                    tm_len = 18
                else:
                    tm_len = 20
                    
                r_primer = revcom(r_temp_left[-tm_len:])
                overhang_r = revcom(r_temp_right[0:rg_ind+24])
                r_rule = 2

            else: #present on both fragments
                join_temp = r_temp_left + r_temp_right
                rg_ind = join_temp.index(curr_guide_2)
                
                if percent_gc(revcom(r_temp_left[-ini_tm_len:])) < 30 :
                    tm_len = 25
                elif percent_gc(revcom(r_temp_left[-ini_tm_len:])) > 70:
                    tm_len = 18
                else:
                    tm_len = 20
                
                r_primer = revcom(r_temp_left[-tm_len:])
                overhang_r = revcom(r_temp_right[0:rg_ind+24-len(r_temp_left)])
                r_rule = 3

            #Tm optimization
            tm_flag = 0
            while tm_flag == 0:
                ta_tm_values = getTM(f_primer,r_primer)
                if np.abs(ta_tm_values[0]-ta_tm_values[1]) > 2:
                    tm_flag = 0
                    tm_len = tm_len + 1 
                    if ta_tm_values[0] - ta_tm_values[1] > 0:
                        #elongation of r primer
                        if r_rule == 1:
                            r_primer = revcom(r_temp_left[rg_ind+24-tm_len:rg_ind+24])
                        elif r_rule == 2:
                            r_primer = revcom(r_temp_left[-tm_len:])
                        elif r_rule == 3:
                            r_primer = revcom(r_temp_left[-tm_len:])

                    else:
                        #elongation of f primer
                        if f_rule == 1:
                            f_primer = f_temp_right[0:tm_len]
                        elif f_rule == 2:
                            f_primer = f_temp_right[fg_ind:fg_ind+tm_len]
                        elif f_rule == 3:
                            f_primer = f_temp_right[0:tm_len]

                else:
                    tm_flag = 1
                    cb_region = np.vstack((cb_region, [i, f_primer, r_primer, np.round(len(sequences[i])*(30/1000),2)]))

            #Overhang addition
            f_start = CntSubstr(cb_region[1][1], sequences[int(cb_region[1][0])])
            f_end = len(sequences[int(cb_region[1][0])]) - CntSubstr(cb_region[1][2], revcom(sequences[int(cb_region[1][0])]))
            PCR_primers = np.vstack((PCR_primers,[cb_region[1][0],overhang_f+cb_region[1][1],overhang_r+cb_region[1][2],cb_region[1][3], len(overhang_f)+len(overhang_r)+len(sequences[int(cb_region[1][0])][f_start:f_end])]))

        #idt file
        plasmid_name = snap_file.split('.')[0]
        idt_sheet = []

        for i in range(np.shape(final_guide)[0]):
            if i == 0:
                idt_name = plasmid_name + '_' + 'G1' + '_j' + str(len(sequences)) + '_' + str(i+1) 
                idt_sheet.append([idt_name,revcom(final_guide[i][1][0:16]),'25 nmole DNA Oligo','Standard Desalting','-','-'])

                idt_name = plasmid_name + '_' + 'G2' + '_j' + str(len(sequences)) + '_' + str(i+1) 
                idt_sheet.append([idt_name,final_guide[i][1][8:24],'25 nmole DNA Oligo','Standard Desalting','-','-'])
            else:
                idt_name = plasmid_name + '_' + 'G1' + '_j' + str(i) + '_' + str(i+1) 
                idt_sheet.append([idt_name,revcom(final_guide[i][1][0:16]),'25 nmole DNA Oligo','Standard Desalting','-','-'])

                idt_name = plasmid_name + '_' + 'G2' + '_j' + str(i) + '_' + str(i+1) 
                idt_sheet.append([idt_name,final_guide[i][1][8:24],'25 nmole DNA Oligo','Standard Desalting','-','-'])

        for i in range(1,np.shape(PCR_primers)[0]):
            idt_name = plasmid_name + '_' + 'FP' + '_' + str(int(PCR_primers[i][0])+1)
            if len(PCR_primers[i][1]) < 60:
                idt_sheet.append([idt_name, PCR_primers[i][1],'25 nmole DNA Oligo','Standard Desalting', PCR_primers[i][3], PCR_primers[i][4]])
            else:
                idt_sheet.append([idt_name, PCR_primers[i][1],'100 nmole DNA Oligo','Standard Desalting', PCR_primers[i][3], PCR_primers[i][4]])

            idt_name = plasmid_name + '_' + 'RP' + '_' + str(int(PCR_primers[i][0])+1)
            if len(PCR_primers[i][2]) < 60:
                idt_sheet.append([idt_name, PCR_primers[i][2],'25 nmole DNA Oligo','Standard Desalting', PCR_primers[i][3], PCR_primers[i][4]])
            else:
                idt_sheet.append([idt_name, PCR_primers[i][2],'100 nmole DNA Oligo','Standard Desalting', PCR_primers[i][3], PCR_primers[i][4]])

        if merge_flag == 1:
            for i in range(1,np.shape(merging_primers)[0]):
                idt_name = plasmid_name + '_' + 'EFP' + '_' + str(int(merging_primers[i][0])+1)
                if len(merging_primers[i][1]) < 60:
                    idt_sheet.append([idt_name, merging_primers[i][1],'25 nmole DNA Oligo','Standard Desalting', merging_primers[i][3], merging_primers[i][4]])
                else:
                    idt_sheet.append([idt_name, merging_primers[i][1],'100 nmole DNA Oligo','Standard Desalting', merging_primers[i][3], merging_primers[i][4]])

                idt_name = plasmid_name + '_' + 'ERP' + '_' + str(int(merging_primers[i][0])+1)
                if len(merging_primers[i][2]) < 60:
                    idt_sheet.append([idt_name, merging_primers[i][2],'25 nmole DNA Oligo','Standard Desalting', merging_primers[i][3], merging_primers[i][4]])
                else:
                    idt_sheet.append([idt_name, merging_primers[i][2],'100 nmole DNA Oligo','Standard Desalting', merging_primers[i][3], merging_primers[i][4]])
        
        idt_sheet_df = pd.DataFrame(idt_sheet, columns =['Name','Sequence','Scale','Purification','Extension Time','Template Length'])
            
        df_index_to_delete = idt_sheet_df[idt_sheet_df.duplicated('Name', keep=False)]
        df_index_to_delete.reset_index(inplace=True)
        index_to_delete = []
        for i in range(np.shape(df_index_to_delete)[0]):
            if i%4 == 1 or i%4 == 2:
                index_to_delete.append(df_index_to_delete['index'][i])

        idt_sheet_df = idt_sheet_df.drop(index_to_delete).reset_index(drop = True)
        idt_sheet_df.to_csv('Primers_Guides_primer3_'+snap_file.split('.')[0]+'.csv', index=False)
        
        for i in range(np.shape(sequences)[0]):
            frag_name_rep = snap_file.split('.')[0] + '_f' + str(i+1)
            frag_seq = sequences[i]
            f_primer = idt_sheet_df['Sequence'][2*np.shape(sequences)[0]+2*i]
            r_primer = idt_sheet_df['Sequence'][2*np.shape(sequences)[0]+2*i+1]

            over_f_flag = 0
            over_r_flag = 0

            if f_primer in frag_seq:
                over_f_flag = 0
            else:
                over_f_flag = 1

            if revcom(r_primer) in frag_seq:
                over_r_flag = 0
            else:
                over_r_flag = 1

            if over_f_flag*over_r_flag == 1:
                inter_frag_seq = reduce(lambda a, b: a + b[overlap(a,b):], [f_primer, frag_seq])
                final_frag_seq = reduce(lambda a, b: a + b[overlap(a,b):], [inter_frag_seq, revcom(r_primer)])

            elif over_f_flag == 1 and over_r_flag == 0:
                inter_frag_seq = reduce(lambda a, b: a + b[overlap(a,b):], [f_primer, frag_seq])
                final_frag_seq = inter_frag_seq[:inter_frag_seq.find(revcom(r_primer))] + revcom(r_primer)

            elif over_f_flag == 0 and over_r_flag == 1:
                inter_frag_seq = reduce(lambda a, b: a + b[overlap(a,b):], [frag_seq, revcom(r_primer)])
                final_frag_seq = inter_frag_seq[inter_frag_seq.find(f_primer):]

            else:
                inter_frag_seq = frag_seq[frag_seq.find(f_primer):]
                final_frag_seq = inter_frag_seq[:inter_frag_seq.find(revcom(r_primer))] + revcom(r_primer)

            final_fragments.append([frag_name_rep, final_frag_seq])
            
        for i in range(len(pre_sequences)-len(sequences)-len(index_to_delete)):
            f_primer_1 = idt_sheet_df['Sequence'][4*np.shape(sequences)[0]+4*i]
            r_primer_1 = idt_sheet_df['Sequence'][4*np.shape(sequences)[0]+4*i+1]
            frag_seq_1 = pre_sequences[int(idt_sheet_df['Name'][4*np.shape(sequences)[0]+4*i].split('EFP_')[1])-1]
            final_frag_seq_1 = reduce(lambda a, b: a + b[overlap(a,b):], [f_primer_1, frag_seq_1, revcom(r_primer_1)])
            final_name_1 = idt_sheet_df['Name'][4*np.shape(sequences)[0]+4*i].split('_')[0] + '_Ext_PCR_f' + idt_sheet_df['Name'][4*np.shape(sequences)[0]+4*i].split('EFP_')[1]

            final_interm_fragments.append([final_name_1, final_frag_seq_1])

            f_primer_2 = idt_sheet_df['Sequence'][4*np.shape(sequences)[0]+4*i+2]
            r_primer_2 = idt_sheet_df['Sequence'][4*np.shape(sequences)[0]+4*i+3]
            frag_seq_2 = pre_sequences[int(idt_sheet_df['Name'][4*np.shape(sequences)[0]+4*i+2].split('EFP_')[1])-1]
            final_frag_seq_2 = reduce(lambda a, b: a + b[overlap(a,b):], [f_primer_2, frag_seq_2, revcom(r_primer_2)])
            final_name_2 = idt_sheet_df['Name'][4*np.shape(sequences)[0]+4*i].split('_')[0] + '_Ext_PCR_f' + idt_sheet_df['Name'][4*np.shape(sequences)[0]+4*i+2].split('EFP_')[1]

            final_interm_fragments.append([final_name_2, final_frag_seq_2])

        pre_verify_re = np.array(['enzyme name','recognition sequence','cut_index','frequency'])
        for re_i in range(np.shape(re_site)[0]):
            curr_re_seq = re_site[re_i,0]
            re_frequency = len([m.start() for m in re.finditer(curr_re_seq, re_plasmid_seq)])
            pre_verify_re = np.vstack((pre_verify_re,[re_site[re_i,3],curr_re_seq,re_site[re_i,2],re_frequency]))    

        verify_re = pre_verify_re[0,:]
        for i in range(1,np.shape(pre_verify_re)[0]):
            if pre_verify_re[i,3] != '0':
                verify_re = np.vstack((verify_re,pre_verify_re[i,:]))

        length_comb = 4 #int(input("Enter number of restriction enzymes you want to use: [Higher number recommended for smaller plasmid]" + "Enter a number less than " + str(np.shape(verify_re)[0]) +":"))
        re_final = ['RE combination','Band Size']   
        comb_to_explore = np.arange(length_comb)
        for p in range(len(comb_to_explore)):
            re_options = []
            re_found = False
            for re_opt_i in itertools.combinations(verify_re[1:,0],p+1):
                re_options.append(np.asarray(re_opt_i))

            re_gel_comb = []
            for l in range(p+1):
                re_gel_comb.append('RE'+ str(l+1))

            re_gel_comb.append('band sizes')
            for i in range(np.shape(re_options)[0]):
                re_names = re_options[i]
                locations = []
                for j in range(len(re_names)):
                    index = np.where(verify_re[:,0] == re_options[i][j])[0][0]
                    matches = re.finditer(verify_re[index,1], re_plasmid_seq)
                    locations.append([match.start() for match in matches])

                re_locations = locations[0]
                if len(locations) > 1:
                    for m in range(len(locations)-1):
                        re_locations = np.concatenate([re_locations,locations[m+1]])

                re_locations = np.sort(re_locations)
                band_size = []
                for j in range(len(re_locations)):
                    if j == len(re_locations) - 1:
                        band_size.append(len(plasmid_seq)-re_locations[j]+re_locations[0])
                    else:
                        band_size.append(re_locations[j+1]-re_locations[j])

                if band_filter(np.sort(band_size)):
                    re_found = True
                    if len(band_size) > 1:
                        band_size_trans = str(';'.join(str(v) for v in list(np.sort(band_size))))
                        re_gel_comb = np.vstack((re_gel_comb,np.concatenate([re_names,[band_size_trans]])))

            if np.array(re_gel_comb).ndim > 1:
                for m in range(np.shape(re_gel_comb)[0]-1):
                    curr_re_comb = ''
                    for k in range(p+1):
                        curr_re_comb = curr_re_comb + re_gel_comb[m+1,k] + ', '

                    re_final = np.vstack((re_final,[curr_re_comb[:-2],re_gel_comb[m+1,np.shape(re_gel_comb)[1]-1]]))

            if p == len(comb_to_explore)-1:
                final_re_df = pd.DataFrame(re_final)
                if final_re_count == 0:
                    common_final_re_df = final_re_df[1:]
                else:
                    common_final_re_df = pd.merge(common_final_re_df, final_re_df[1:], how='inner', on = 0)

                final_re_count = 1
                common_final_re_df_index.append(snap_file.split('.')[0])

if working_plasmid_count > 0:
    common_final_re_df.columns = common_final_re_df_index
    common_final_re_df.to_csv('Combined_RE_for_verification'+'.csv', index=False)

    final_frag_df = pd.DataFrame(final_fragments, columns = ['name', 'seq']).groupby('seq')['name'].apply(lambda x: x.unique()).reset_index()
    final_frag_df[final_frag_df['name'].map(len) > 1]['name'].reset_index(drop=True).to_csv('Repeated_Fragments'+'.csv', index=True)

    interm_df = pd.DataFrame(final_interm_fragments, columns = ['name', 'seq']).groupby('seq')['name'].apply(lambda x: x.unique()).reset_index()
    interm_df[interm_df['name'].map(len) > 1]['name'].reset_index(drop=True).to_csv('ext_PCR_Repeated_Fragments'+'.csv', index=True)

All restriction enzymes are palindromic
Use wild-type + mutant PfAgo for plasmid assembly
Guides obtained successfully for 17kb-17p_modified.dna
